<a href="https://colab.research.google.com/github/kate-431/MGprogramming_task/blob/main/Mantelgroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import json 
from google.colab import drive
from google.colab import files
import glob
import os
from collections import Counter

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip drive/My\ Drive/Interview/drive-download-20220810T054936Z-001.zip

Archive:  drive/My Drive/Interview/drive-download-20220810T054936Z-001.zip
  inflating: 5.json                  
  inflating: 2.json                  
  inflating: 4.json                  
  inflating: 0.json                  
  inflating: 3.json                  
  inflating: 6.json                  
  inflating: 1.json                  


In [4]:
#Create one dataset from all json files
temp = pd.DataFrame()

path_to_json = '/content/' 

json_pattern = os.path.join(path_to_json,'*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
    dfs.append(json_data)
df = pd.concat(dfs, sort=False)

In [5]:
df.head()

,LocalObservationDateTime,EpochTime,WeatherText,WeatherIcon,HasPrecipitation,PrecipitationType,IsDayTime,city,Temperature.Metric.Value,Temperature.Metric.Unit,Temperature.Metric.UnitType,Temperature.Imperial.Value,Temperature.Imperial.Unit,Temperature.Imperial.UnitType
0,2022-08-04T11:25:00+09:30,1659578100,MOStlY CloUdy,6,False,None,True,darwIN,26.1,C,17,79.0,F,18
1,2022-08-04T10:25:00+09:30,1659574500,paRtLY SUNNY,3,False,None,True,dArWin,26.4,C,17,80.0,F,18
2,2022-08-04T09:25:00+09:30,1659570900,SunNy,1,False,None,True,dARwiN,25.2,C,17,77.0,F,18
3,2022-08-04T08:25:00+09:30,1659567300,SUNnY,1,False,None,True,daRwIn,24.8,C,17,77.0,F,18
4,2022-08-04T07:25:00+09:30,1659563700,suNny,1,False,None,True,DARwin,24.2,C,17,76.0,F,18


In [6]:
#Check data for NA or null values in the temperature value columns and replace with mean if value is missing.
def clean_data(df):
   for i in df["Temperature.Metric.Value"]:
    if pd.isna(i):
     df = df.fillna(df["Temperature.Metric.Value","Temperature.Imperial.Value"].mean())
     print("NA and Null values replaced with mean value for Temperature.Metric.Value/Temperature.Imperial.Value variables")
   else: 
       print("No NA or Null values present.")
   print("")
#Make all strings lower case in dataset
   df= df.applymap(lambda s:s.lower() if type(s) == str else s) 

   return df


In [7]:
df =clean_data(df)

No NA or Null values present.



In [8]:
#Function to find Average temperature.
def Average_temperature(df):
  Avg_temp = df.groupby('city', sort=False).agg(Average_Celcius=('Temperature.Metric.Value','mean'),
                                                Average_Fahrenheit=('Temperature.Imperial.Value','mean')).reset_index(drop=False)
  return Avg_temp 

In [9]:
#Function to calculate top 3 WeatherText by city. 
#Solution takes into account that some citys may not have 3 different elements for top 3 or may have a tie between elements.
def get_nlargest(df, n, keep):
    top_n = df.value_counts(normalize=True).nlargest(n, keep)
    other = pd.Series({'other': 1 - top_n.sum()})
    return pd.concat([top_n, other])

def weather_nlargest(df, n, keep='all'):
    df=df.groupby(['city'])['WeatherText'].apply(get_nlargest, n, keep).reset_index()
    df.rename(columns = {'city':'City', 'level_1':'WeatherText', 'WeatherText':'WeatherType_%'}, inplace = True)
    df["WeatherType_%"] = round(df["WeatherType_%"],2)
    print("If there is a tie between WeatherText elements, the output will return more than 3 elements.")
    return df


In [10]:
Avg_temp = Average_temperature(df)
Avg_temp.to_csv('Average_temp.csv', index=False) 
files.download('Average_temp.csv')
print("Average temperature sucessfully downloaded as CSV.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Average temperature sucessfully downloaded as CSV.


In [11]:
weather_text_top3 = weather_nlargest(df,n=3)

weather_text_top3.to_csv('Weather_text_top3.csv', index=False) 
files.download('Weather_text_top3.csv')
print("Top 3 WeatherType sucessfully downloaded as CSV.")

If there is a tie between WeatherText elements, the output will return more than 3 elements.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Top 3 WeatherType sucessfully downloaded as CSV.
